In [5]:
from fitting import get_model, get_model_name, evaluate
from dataprocessing import generate_tasks
from meta import Meta
import numpy as np
import torch

# Load the model

In [2]:
# Model architecture
hidden_layers = [112, 90, 95, 33]

# Arguments to create model
sup_shots = 100
que_shots = 200
name = get_model_name(sup_shots, que_shots, hidden_layers)
config = get_model(hidden_layers)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Model filename
filename = "models/" + name + ".pt"

# Choose PyTorch device and load the model
model = Meta(name, config, sup_shots, que_shots, device).to(device)
model.load_params(filename)

# Load the data

In [6]:
# Datapath and background file for data-files
datapath = "processed-data/"
bkg_file = datapath + "bkg.h5"

# Signal files for each task split
val_signals = ["hg3000_hq1200", "wohg_hq1000"]
test_signals = ["wohg_hq1400", "fcnc"]

# Add datapath and extention to files for each split
val_signals = [datapath + p + ".h5" for p in val_signals]
test_signals = [datapath + p + ".h5" for p in test_signals]

# Generate tasks
val_tasks = generate_tasks(val_signals, bkg_file, args["k_sup"], args["k_que"])
test_tasks = generate_tasks(test_signals, bkg_file, args["k_sup"], args["k_que"])

# Evaluate on the val and test data

In [ ]:
evaluate(model, val_tasks, 200, "Eval Val")